# Break repeating-key XOR
### It is officially on, now.
This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.

There's a file here. It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
2. Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
```
this is a test
```
and
```
wokka wokka!!!
```
is 37. Make sure your code agrees before you proceed.
3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
7. Solve each block as if it was single-character XOR. You already have code to do this.
8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.

### No, that's not a mistake.
We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.

In [1]:
from cryptopals import *

In [40]:
def hamming_distance(b1, b2):
    '''Finds the bit-wise edit distance / Hamming distance between two byte arrays'''
    xor_bit_string = ''.join(['{0:b}'.format(b) for b in fixed_xor(b1, b2)])
    return sum([c == '1' for c in xor_bit_string])

def hamming_distance_str(s1, s2):
    '''Finds the bit-wise edit distance / Hamming distance between two ascii-encoded strings'''
    return hamming_distance(bytearray(s1), bytearray(s2))

In [37]:
?reduce

In [28]:
t1 = 'this is a test'
t2 = 'wokka wokka!!!'

In [29]:
fixed_xor(bytearray(t1), bytearray(t2))

bytearray(b'\x03\x07\x02\x18AI\x04O\nK\x15DRU')

In [35]:
sum([sum(map(lambda c: c=='1', '{0:b}'.format(b))) for b in fixed_xor(bytearray(t1), bytearray(t2))])

37

In [31]:
print list(bytearray(t1))
print list(bytearray(t2))

[116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116]
[119, 111, 107, 107, 97, 32, 119, 111, 107, 107, 97, 33, 33, 33]


In [34]:
map(lambda c: c=='1', '{0:b}'.format(73))

[True, False, False, True, False, False, True]

In [41]:
hamming_distance_str(t1, t2)

37